In [ ]:
 # TODO: 1) Analizar si está ok el simulate user choice y si es conveniente escalar el wine_score, 2) simulate like

# synthetic_user_simulator.py
import random
import sys
import os
import logging
from typing import Dict, Tuple, List, Union

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from scipy import stats

sys.path.append(os.path.abspath(os.path.join('..', '..', 'src', 'utils')))
import utils as ut

class SyntheticUserSimulator:
    """
    Esta clase simula usuarios que eligen vinos basados en preferencias sintéticas
    y evalúan si les gustó o no una recomendación.

    Objetivo: generar datasets sintéticos para entrenar los modelos de Machine Learning.
    """

    def __init__(self, wine_df: pd.DataFrame, meals_df: pd.DataFrame,
                 pairing_cols: List[str], taste_cols: List[str], grape_cols: List[str],
                 weights: Dict[str, float], top_n: int = 20):
        """
        Constructor: inicializa la clase con los datos base.

        Args:
        - wine_df: DataFrame de vinos.
        - meals_df: DataFrame de recetas/platos.
        - pairing_cols: Lista con nombre de las columnas a utilizar como pairing.
        - taste_cols: Lista con nombre de las columnas a utilizar como taste.
        - weights: Diccionario con pesos para cada componente del score (rating, price_quality, rating_qty, user_similarity, main_pairing).
        - top_n: Cantidad máxima de vinos que el usuario considera para elegir uno.
        """
        self.meals_df = meals_df.copy()
        self.pairing_cols = pairing_cols
        self.taste_cols = taste_cols
        self.grape_cols = grape_cols
        self.weights = weights
        self.top_n = top_n
        self.mm_scaler = MinMaxScaler()
        self.robust_scaler = RobustScaler()
        self.wine_df = wine_df

        # Pre-transformación de df (escalado tastes + agrega "otras uvas")
        self.tra_df, self.taste_cols_scld = self._transform_df()

        # Perfiles de pairing (cuantiles y stds por pairing y taste)
        self._taste_profiles, self._taste_deviations = self._build_pairing_profile()

    def generate_user_input(self) -> Dict:
        """
        Genera un input sintético de usuario (gustos, pairing, uvas, etc.)

        Returns:
        - Diccionario con los parámetros preferidos por el usuario.
        """
        selected_meal = self._select_random_meal()
        selected_grapes = self._select_grapes()
        selected_price_range = self._select_price_range()
        selected_profile = self._select_taste_profile(selected_meal.get("main_pairing"))

        user_input = {
            "meal": selected_meal["meal"],
            "main_pairing": selected_meal["main_pairing"],
            "pairing_list": selected_meal["pairing_list"],
            "grape_list": selected_grapes,
            "precio_min": selected_price_range[0],
            "precio_max": selected_price_range[1],
            "tastes": {
                "body": selected_profile["body_scld"][1],
                "tannins": selected_profile["tannins_scld"][1],
                "sweetness": selected_profile["sweetness_scld"][1],
                "acidity": selected_profile["acidity_scld"][1]
            },
            "weights": self.weights
        }

        return user_input


    def _build_pairing_profile(self, quantiles=[0, .25, .5, .75, 1]) -> Tuple[Dict, Dict]:
        """
        Crea los perfiles de sabor por cuantiles por pairing.
        - Objetivo: que haga sentido el perfil elegido con el seleccionado por el user.

        Args:
        - quantiles: Lista con cuantiles a considerar.

        Returns:
        - profile: Diccionario con cuantiles por sabor y pairing.
        - deviations: Diccionario con desvíos estándar por sabor y pairing.
        """
        profiles = {}
        deviations = {}
        for pairing in self.pairing_cols:
            pairing_subset = self.tra_df[self.tra_df[pairing] == 1]
            profiles[pairing] = {}
            deviations[pairing] = {}
            for taste in self.taste_cols_scld:
                # Cuantiles
                taste_quantile = pairing_subset[taste].quantile(quantiles)
                profiles[pairing][taste] = taste_quantile

                # Desvío Estandar (para distribución normal en select_taste_profile)
                std = pairing_subset[taste].std()
                deviations[pairing][taste] = std

        return profiles, deviations
    

    def _select_random_meal(self) -> Dict[str, object]:
        """
        Selecciona una comida al azar y devuelve info útil para construir el perfil del usuario.

        Returns:
        - Dict con:
            - complete_meal (pd.Series): la comida completa (ingredientes, columnas one-hot).
            - meal (str): el nombre de la comida.
            - main_pairing (str): ingrediente principal en lowercase.
            - pairing_list (List[str]): lista de pairings que tienen valor 1.
        """
        i = np.random.randint(0, len(self.meals_df))
        complete_meal = self.meals_df.loc[i]

        meal = complete_meal["Comida"]

        main_pairing = complete_meal["Ingrediente Principal"].lower()

        # Excluye las 2 primeras columnas: "Comida" e "Ingrediente Principal"
        one_hot = complete_meal[2:]
        pairing_list = list(one_hot.index[one_hot > 0])

        return {
            "complete_meal": complete_meal,
            "meal": meal,
            "main_pairing": main_pairing,
            "pairing_list": pairing_list
        }
    
    
    def _select_taste_profile(self, main_pairing: str, prob: float = 0.2,
                          categories: List[str] = ["leve", "moderado", "marcado", "intenso"]) -> Dict[str, str]:
        """
        Selecciona un perfil de gustos de vino en base al pairing principal.
        Usa lógica probabilística con distribución normal y categorías definidas (sentido común + random).

        Returns:
        - Diccionario con el gusto preferido por cada variable de sabor (body, sweetness, etc.)
        """
        selected_profile = {}
        for taste in self.taste_cols_scld:
            quantiles = self._taste_profiles[main_pairing][taste]
            std = self._taste_deviations[main_pairing][taste]

            if random.random() < prob:
                # Opción aleatoria entre cuantiles medios
                quant_random_options = [
                    (quantiles.values[i] + quantiles.values[i+1])/2
                    for i in range(len(quantiles.values)-1)
                ]
                selected_val = np.random.choice(quant_random_options)
            else:
                median = quantiles[.5]
                min_val = quantiles[0]
                max_val = quantiles[1]
                selected_val = min(max(np.random.normal(loc=median, scale=std), min_val), max_val)
            
            selected_profile[taste] = self._get_category(selected_val, quantiles, categories)
        return selected_profile


    def _get_category(self, value, quantile_values: pd.Series,
                      categories: List[str] = ["leve", "moderado", "marcado", "intenso"]) -> Tuple[str, List[float]]:
        """
        Clasifica un valor en una categoría sensorial (leve, moderado, etc.) según cuantiles.

        Args:
        - value: valor a clasificar (escalado)
        - quantile_values: serie de cuantiles para esa variable sensorial
        - categories: etiquetas para cada rango de cuantiles

        Returns:
        - Tuple (categoría asignada, rango de valores del cuantil correspondiente)
        """
        if len(quantile_values)-1 != len(categories):
            raise IndexError("Length of quantiles ranges and categories does not match!")
        
        quant_values = quantile_values.values

        # Obtiene categoría y valor promedio del rango de los cuantiles
        for i in range(len(quant_values)-1):
            start_val = quant_values[i]
            end_val = quant_values[i+1]
            if start_val <= value < end_val:
                return categories[i], [start_val, end_val]
        
        # Gestiona valores atípicos (mayores que el máximo valor de los cuantiles)
        if value >= quant_values[-1]:
            return quant_values[-1], [quant_values[-2], quant_values[-1]]
        
        # Gestiona valores atípicos (menores que el mínimo valor de los cuantiles)
        if 0 <= value < quant_values[0]:
            return categories[0], [quant_values[0], quant_values[1]]
        
        # Error si el valor es inferior a cero
        raise ValueError(f"Passed value is less than zero! {value}")


    def _select_grapes(self,
                       top_n_grapes: int = 8,
                       min_grapes: int = 1,
                       max_grapes: int = 4) -> List[str]:
        """
        Selecciona aleatoriamente un set de uvas según su frecuencia en el dataset.

        Args:
        - top_n_grapes: cuántas uvas populares considerar (resto = Otras Uvas).
        - min_grapes: cantidad mínima de uvas a seleccionar.
        - max_grapes: cantidad máxima de uvas a seleccionar.

        Returns:
        - Lista de uvas seleccionadas.
        """
        grapes_df = self.wine_df[self.grape_cols]
        top_grapes = self._get_top_grapes(grapes_df, top_n_grapes)
        total_grapes_in_df = sum(grapes_df.sum(axis=0))

        top_grapes_prob = top_grapes / total_grapes_in_df
        top_grapes_prob["Otras Uvas"] = 1 - sum(top_grapes_prob)
        grapes_prob = dict(top_grapes_prob)

        while True:
            grape_selection = [grape for grape, prob in grapes_prob.items() if random.random() < prob]
            if len(grape_selection) >= min_grapes:
                break
            
        return grape_selection[:max_grapes]
    

    def _get_top_grapes(self, grapes_df: pd.DataFrame, top_n_grapes: int) -> pd.Series:
        """
        Devuelve las top N uvas más frecuentes del dataset.

        Args:
        - grapes_df: DataFrame con columnas de uvas (one-hot).
        - top_n_grapes: número de uvas más frecuentes a devolver.

        Returns:
        - Serie con el total de apariciones por uva el el DataFrame (orden descendente).
        """
        top_grapes = grapes_df.sum(axis=0).sort_values(ascending=False).head(top_n_grapes)
        return top_grapes

    def _select_price_range(
            self,
            quantiles: List[float] = [.075, .125, .25, .375, .5, .625, .75, .875],
            low_prices_weights: List[float] = [.2, .3, .25, .15, .05, .025, .015, 0.01, 0, 0],
            min_dispersion: int = 2
    ) -> List[Union[int, None]]:
        """
        Selecciona un rango de precios realista con sesgo hacia precios bajos, simulando comportamiento de usuarios reales.

        Args:
            - quantiles -> cuantiles a considerar para definir posibles precios.
            - low_prices_weights -> pesos para elegir precios bajos (más probabilidad al principio).
            - min_dispersion -> distancia mínima entre el precio bajo y el alto en el rango de cuantiles.

        Returns:
            - Lista [min_price, max_price], donde max_price puede ser None si no hay límite superior.
        """
        df = self.wine_df

        # Eliminación de outliers (IQR * 1.5)
        no_outlier_price = ut.manage_outlier_IQR(df=df["price"], func="remove")
        price_quantiles = no_outlier_price.quantile(quantiles)

        # Adición de extremos (sin precio mínimo / máximo)
        price_quantiles[0], price_quantiles[1] = 0, -1
        price_quantiles = price_quantiles.sort_index()
        price_quantiles = pd.Series(price_quantiles.values)

        # Selección índice de cuantil de precio inferior
        low_price_index = random.choices(population=range(len(price_quantiles)), weights=low_prices_weights, k=1)[0]

        # Cálculo de límite superior posible
        high_top_limit = len(price_quantiles)
        high_bottom_limit = min(low_price_index + min_dispersion, high_top_limit-1)
        high_prices_possible = list(range(high_bottom_limit, high_top_limit))
        high_prices_weights = [1 / x**1.5 for x in high_prices_possible]

        # Selección de índice de cuantil de precio superior
        high_price_index = random.choices(population=high_prices_possible, weights=high_prices_weights, k=1)[0]

        # Rango de precios
        price_range = [int(price_quantiles[low_price_index]), int(price_quantiles[high_price_index])]
        if price_range[1] == -1:
            price_range[1] = None

        return price_range


    def _transform_df(self) -> Tuple[pd.DataFrame, List]:
        """
        Pre-transforma el DataFrame para la simulación.

        Returns:
        - DataFrame con:
            1. Tastes escalados (body, tannins, etc.)
            2. Columna "Otras Uvas".
            3. Métrica de calidad-precio global.
        - Lista con nombre de columnas de sabor escaladas.
        """
        tra_df = self.wine_df.copy()
        tra_df, taste_cols_scld = self._df_add_scaled_tastes(tra_df)  # Escala tastes
        tra_df = self._df_group_other_grapes(tra_df)                  # Agrupa Otras Uvas
        tra_df = self._df_add_qual_price(tra_df)                      # Agrega Métrica Calidad-Precio
        return tra_df, taste_cols_scld


    def _df_add_scaled_tastes(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, List]:
        """
        Escala columnas de sabor del dataframe (body, tannins, etc.)

        Args:
        - df: DataFrame con las columnas taste a escalar.

        Returns:
        - DataFrame con columnas de sabor Min-Max scaled (sufijo '_scld').
        - Lista con nombres de columnas de sabor escaladas.
        """
        tra_df = df.copy()
        taste_cols_scld = [taste + "_scld" for taste in self.taste_cols]
        tra_df[taste_cols_scld] = self.mm_scaler.fit_transform(tra_df[self.taste_cols])
        return tra_df, taste_cols_scld

    def _df_group_other_grapes(self, df: pd.DataFrame, top_n_grapes: int = 8) -> pd.DataFrame:
        """
        Agrega agrupa todas las uvas que no son 'top wines' en la columna 'Otras Uvas'.
        
        Args:
        - df: el DataFrame con las columnas de uvas a agrupar.
        - top_n_grapes: número de uvas más frecuentes no agrupadas en 'Otras Uvas'.

        Returns:
        - Dataframe con uvas fuera del top_n_wines agrupadas en la columna 'Otras Uvas'.
        """
        tra_df = df.copy()
        grapes_df = tra_df[self.grape_cols]
        top_grapes = list(self._get_top_grapes(grapes_df, top_n_grapes).index)
        other_grapes = [grape for grape in self.grape_cols if grape not in top_grapes]
        tra_df["Otras Uvas"] = tra_df[other_grapes].max(axis=1)
        tra_df = tra_df.drop(columns=other_grapes)
        return tra_df

    
    # Se aplica 2 veces: 1) Global, 2) Local (por cada user)
    def _df_add_qual_price(self, df: pd.DataFrame) -> pd.DataFrame:
        # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html
        """
        Agrega métricas de precio calidad al DataFrame.

        Args:
            df -> DataFrame al cual agregar métricas de precio calidad.
        
        Returns:
            - DataFrame con nuevas columnas:
                - rating_rscld -> rating con escalado robusto.
                - price_rscld -> precio con escalado robusto.
                - quality_price -> diferencia entre rating y precio escalados.
                - quality_price_rscld -> métrica de precio calidad entre 0 y 1 con escalado robusto.
        """
        tra_df = df.copy()
        # Cálculo de zscore para rating y price (mediana = 0, std = 1)
        rscaler = self.robust_scaler
        tra_df[['rating_rscld', 'price_rscld']] = rscaler.fit_transform(tra_df[["rating", "price"]])
                
        # Cálculo de métrica calidad-precio (calidad suma, precio resta)
        tra_df["quality_price"] = tra_df['rating_rscld'] - tra_df['price_rscld']
        
        # Escalado robusto con outliers fuera del fit, pero incluidos en el transform 
        no_outliers = ut.manage_outlier_IQR(
            tra_df['quality_price'], i=2, func="remove"
        )

        mod_mm_scaler = self.mm_scaler.fit(no_outliers.to_frame())

        # TODO: agregar visualización de distribución en el EDA

        # Límite de quality price en 0 y 1 (capear los outliers)
        scld_qual = np.clip(mod_mm_scaler.transform(tra_df[["quality_price"]]), a_min=0, a_max=1)
        tra_df["quality_price_rscld"] = scld_qual

        return tra_df


    def _score_wines(self, df: pd.DataFrame, user_input: Dict) -> pd.DataFrame:
        """
        Asigna un score a cada vino según el perfil del usuario.

        Args:
        - df: DataFrame sobre el cual calcular el wine_score.
        - user_input: Diccionario con gustos y preferencias del usuario.

        Returns:
        - DataFrame con vinos, sus scores y sus scores escalados.
        """
        tra_df = df.copy()

        # Cálculo de similitud de gustos del user contra cada vino
        user_tastes = user_input["tastes"]
        tra_df = self._df_add_user_similarity(tra_df, user_tastes)

        # Obtención de weights
        user_weights = user_input["weights"]
        rating_w = user_weights["rating"]
        price_quality_w = user_weights["price_quality"]
        rating_qty_w = user_weights["rating_qty"]
        user_similarity_w = user_weights["user_similarity"]
        main_pairing_w = user_weights["main_pairing"]

        # Puntaje sintético del vino
        tra_df["wine_score"] = (
            rating_w * (tra_df["rating"] / 5) + 
            price_quality_w * tra_df["quality_price_rscld"] + 
            rating_qty_w * (tra_df["rating_qty"] / tra_df["rating_qty"].max()) +
            user_similarity_w * tra_df["user_similarity"] + 
            main_pairing_w * tra_df[user_input["main_pairing"]]
        )

        # Min-Max Scaling del score
        tra_df["wine_score_scld"] = self.mm_scaler.fit_transform(tra_df[["wine_score"]])

        return tra_df


    def _df_add_user_similarity(self, df: pd.DataFrame, tastes: Dict) -> pd.DataFrame:
        """
        Agrega columna de similitud con el input del usuario al df.

        Args:
            - df -> Dataframe al que agregar la columna 'user_similarity'
            - tastes -> Diccionario con gustos y preferencias del usuario.

        Returns:
            - Dataframe con columna 'user_similarity' agregada.
        """

        tra_df = df.copy()
        # Calcula la similitud con los gustos del usuario con Fuzzy Distance
        distances = self._compute_fuzzy_distance(tra_df[self.taste_cols_scld], tastes)
        tra_df["user_similarity"] = 1 - (distances / distances.max())
        return tra_df
        

    def _compute_fuzzy_distance(self, tastes_df: pd.DataFrame, tastes: Dict) -> np.array:
        """
        Computa la distancia tipo fuzzy entre los gustos del usuario y los vinos disponibles.

        Args:
        - tastes_df: DataFrame de columnas taste escaladas.
        - tastes: Diccionario con gustos y preferencias del usuario.

        Returns:
        - Numpy Array con las distancias entre cada registro del DataFrame y los gustos del usuario.
        """

        # Cálculo de distancias por cada vino (registro)
        distances = []
        for _, row in tastes_df.iterrows():
            wine_distance = 0
            col_count = 0
            # Calculo de Fuzzy Distance de cada taste (columna)
            for col in tastes_df.columns:
                key = col.replace("_scld", "")
                if key in tastes:
                    value = row[col]
                    q_low, q_high = tastes[key]
                    wine_distance += self._fuzzy_distance(value, q_low, q_high)
                    col_count += 1
            
            # Cálculo del promedio de distancias
            distance = wine_distance / col_count if col_count > 0 else None
            distances.append(distance)

        distances = np.array(distances)
        return distances
    

    def _fuzzy_distance(self, value: Union[float, int], q_low: Union[float, int], q_high: Union[float, int]) -> Union[float, int]:
        """
        Calcula la Fuzzy Distance entre un valor y el rango de valores dados.

        Args:
        - value: valor del gusto del usuario.
        - q_low: límite inferior del rango del gusto.
        - q_high: límite superior del rango del gusto.

        Return:
        - Valor de la Fuzzy Distance entre el valor y el rango ingresado
        """
        # Distancia escalada contra límite inferior
        if value < q_low:
            return (q_low - value) / (q_high - q_low)
        # Distancia escalada contra límite superior
        elif value > q_high:
            return (value - q_high) / (q_high - q_low)
        # Valor dentro del rango (distancia = 0)
        else:
            return 0.0


    def simulate_user_choice(self, scored_df: pd.DataFrame, user_input: Dict, top_n: int=20, d: float=.075) -> Union[pd.Series, None]:
        """
        Elige los top_n vinos con mayor score.

        Parameters:
            - scored_df -> DataFrame con vinos y sus scores globales calculados.
            - user_input -> Diccionario con selección del usuario.

        Returns:
            - Serie con datos del vino seleccionado (None si no hay vinos para los parámetros del user).
        """
        # User Inputs
        user_pairings = user_input.get("pairing_list")
        precio_min = user_input.get("precio_min")
        precio_max = user_input.get("precio_max")
        grape_list = user_input.get("grape_list")
        user_tastes = user_input.get("tastes")

        # Renombramiento de variables del scored_df como Globales (consideran toda la data)
        cols_to_rename = ['rating_rscdl', 'price_rscdl', 'quality_price','quality_price_rscld',
                          'user_similarity', 'wine_score', 'wine_score_scld']
        renamed_cols = [col_name + "_gbl" for col_name in cols_to_rename]
        renaming_dict = dict(zip(cols_to_rename,renamed_cols))
        wine_base = scored_df.rename(columns=renaming_dict).copy() # Utilizada para cálculo local ("por user")
        wine_base_gbl = wine_base.copy() # Copia global
        # TODO: MEJORAR ESTO PARA QUE NO CREE LA COPIA GLOBAL EN CADA CORRIDA

        # Filtro por pairings
        if user_pairings is not None:
            wine_base = wine_base[wine_base[user_pairings].sum(axis=1)>0]

        # Filtro por precio
        if precio_min is not None:
            wine_base = wine_base[wine_base["price"]>=precio_min]
        if precio_max is not None:
            wine_base = wine_base[wine_base["price"]<=precio_max]
        
        # Filtro por uvas
        if grape_list is not None:
            wine_base = wine_base[wine_base[grape_list].sum(axis=1)>0]

        # Quality/price local
        wine_base = self._df_add_qual_price(wine_base)

        # Solo se ejecuta si existen vinos para la selección del usuario
        if len(wine_base) > 0:
            # User similarity (Fuzzy Distance)
            wine_base = self._df_add_user_similarity(wine_base, user_tastes)

            # Puntaje sintético local del vino
            wine_base = self._score_wines(wine_base, user_input) # Puntaje "crudo"

            if len(wine_base) > max(top_n//2, 10):
                # Si el wine_base es lo suficientemente grande, el puntaje es escalado considerando la selección de filtros del usuario
                wine_base["wine_score_scld"] = self.mm_scaler.fit_transform(wine_base[["wine_score"]])
            else:
                # Si el wine_base es pequeño, se considera el wine_score global (sin filtros de usuario)
                if wine_base.index.isin(wine_base_gbl.index).all():
                    wine_base["wine_score_scld"] = wine_base_gbl.loc[wine_base.index]["wine_score_scld_gbl"]
                else:
                    raise IndexError("Index in wine_base and global dataframe passed doesn't match!")


            # Selección de uno de los mejores top_n vinos
            top_wine_base = wine_base.nlargest(top_n, "wine_score")
            top_selection_weights = [(1 - d)**n for n in range(len(top_wine_base))]    # Pesos decrecientes de a d%
            top_selection_weights = [w*random.random() for w in top_selection_weights] # Agregamos factor aleatorio con suma positiva
            total_weight = sum(top_selection_weights)
            top_selection_weights = [w / total_weight for w in top_selection_weights]  # Normalizamos para interpretabilidad
            selected_wine_id = int(random.choices(top_wine_base.index, top_selection_weights, k=1)[0]) # Selección de 1 vino según vector de pesos
            selected_wine = wine_base.loc[selected_wine_id] # Localización de vino por id en la base de vinos
            return selected_wine, top_wine_base # TODO: quitar top_wine_base
        # Se ejecuta si no existen vinos para la selección del usuario
        else:
            logging.info("No wines match user input!")
            return None
    
    def simulate_like(self, wine: pd.Series, user_profile: Dict) -> int:
        """
        Determina si al usuario le gustó o no un vino en base al score y desviación.

        Parameters:
        - wine: Serie con info del vino.
        - user_profile: Diccionario del perfil del usuario.

        Returns:
        - 1 si le gustó, 0 si no.
        """
        pass  # <-- Tu lógica de simulate_like va acá

    def generate_synthetic_data(self, n_users: int) -> pd.DataFrame:
        """
        Genera datos sintéticos para n usuarios.

        Parameters:
        - n_users: Cantidad de usuarios sintéticos a simular.

        Returns:
        - DataFrame con columnas: user_id, wine_id, wine_score, liked
        """
        synthetic_data = []

        for i in range(n_users):
            user_profile = self.generate_user_profile()
            scored_wines = self.score_wines(user_profile)
            top_wines = self.simulate_user_choice(scored_wines)

            for _, row in top_wines.iterrows():
                liked = self.simulate_like(row, user_profile)
                synthetic_data.append({
                    'user_id': f'user_{i}',
                    'wine_id': row['wine_id'],
                    'wine_score': row['wine_score'],
                    'liked': liked
                })

        return pd.DataFrame(synthetic_data)

In [332]:
# Class Main Inputs
wines_df = pd.read_csv("../../src/data/transformed/wines_clean.csv")
meals_df = pd.read_excel("../../src/data/raw/meals/Meals.xlsx")
pairings = pd.read_csv("../../src/data/processed/aux/pairings.csv")
pairings = list(pairings["pairings"])
taste_columns = ["body", "tannins", "sweetness", "acidity"]
simulation_weights = {
    "rating": .25,
    "price_quality": .25,
    "rating_qty": .1,
    "user_similarity": .3,
    "main_pairing": .1
}

# Grape Methods Input
grapes = pd.read_csv("../../src/data/processed/aux/grapes.csv")
grapes = list(grapes["grapes"])

user = SyntheticUserSimulator(
    wine_df=wines_df, meals_df=meals_df, pairing_cols=pairings, taste_cols=taste_columns, grape_cols=grapes, weights=simulation_weights)

user_input = user.generate_user_input()
tra_df = user.tra_df
tra_df = user._df_add_qual_price(tra_df)
tra_df = user._score_wines(tra_df, user_input)
#user.simulate_user_choice(tra_df, user_input)

In [ ]:
selection, top_base = user.simulate_user_choice(tra_df, user_input)
display_columns = [
    'name',
 'year',
 'winery',
 'rating',
 'rating_qty',
 'price',
 'rating_rscld',
 'price_rscld',
 'quality_price_gbl',
 'quality_price_rscld_gbl',
 'user_similarity_gbl',
 'wine_score_gbl',
 'wine_score_scld_gbl',
 'quality_price',
 'quality_price_rscld',
 'user_similarity',
 'wine_score',
 'wine_score_scld'
 ]
selection[display_columns]

    # TODO: PENSAR SI ES CONVENIENTE O NO ESCALAR EL WINE_SCORE

name                       Reserve Malbec
year                               2023.0
winery                          Cruz Alta
rating                                4.1
rating_qty                          250.0
price                                13.0
rating_rscld                          1.0
price_rscld                     -0.880459
quality_price_gbl                 0.79746
quality_price_rscld_gbl          0.842615
user_similarity_gbl              0.865447
wine_score_gbl                    0.77667
wine_score_scld_gbl              0.857501
quality_price                    1.880459
quality_price_rscld                   1.0
user_similarity                  0.745523
wine_score                       0.780039
wine_score_scld                  0.944213
Name: 1534, dtype: object

In [334]:
top_base[display_columns]

,name,year,winery,rating,rating_qty,price,rating_rscld,price_rscld,quality_price_gbl,quality_price_rscld_gbl,user_similarity_gbl,wine_score_gbl,wine_score_scld_gbl,quality_price,quality_price_rscld,user_similarity,wine_score,wine_score_scld
1363,Signature Rosé,2019.0,Susana Balbo,4.2,619.0,18.37,1.333333,0.067049,0.923097,0.870654,0.960233,0.819155,0.938767,1.266284,0.852074,0.924790,0.803877,1.000000
411,Valle Las Acequias Malbec Oak,2021.0,Luis Segundo Correas,4.1,26.0,15.99,1.000000,-0.352889,0.681815,0.816806,0.938907,0.791017,0.884946,1.352889,0.872933,0.884457,0.788714,0.964515
1437,Malbec,2019.0,Catena,4.1,18091.0,20.99,1.000000,0.529334,0.488429,0.773647,0.842518,0.851167,1.000000,0.470666,0.660446,0.702158,0.780759,0.945898
1534,Reserve Malbec,2023.0,Cruz Alta,4.1,250.0,13.00,1.000000,-0.880459,0.797460,0.842615,0.865447,0.776670,0.857501,1.880459,1.000000,0.745523,0.780039,0.944213
1858,Malbec,2022.0,Séptima,3.9,582.0,10.50,0.333333,-1.321570,0.227487,0.715412,0.885577,0.742743,0.792607,1.654904,0.945674,0.783594,0.769714,0.920050
1727,Reserva Malbec,2020.0,Finca el Origen,4.0,112.0,12.99,0.666667,-0.882223,0.464514,0.768310,0.887512,0.758950,0.823608,1.548890,0.920140,0.787255,0.766831,0.913302
1770,Trumpeter Malbec,2020.0,Rutini,3.9,4212.0,11.99,0.333333,-1.058668,0.169858,0.702551,0.858079,0.751344,0.809058,1.392001,0.882353,0.731588,0.758347,0.893448
1693,Malbec,2023.0,Ed Edmundo,4.0,252.0,11.97,0.666667,-1.062197,0.503964,0.777114,0.851014,0.750976,0.808354,1.728863,0.963488,0.718227,0.757733,0.892011
1766,Malbec,2021.0,Alamos,3.9,9302.0,15.99,0.333333,-0.352889,0.015149,0.668024,0.881750,0.777949,0.859948,0.686223,0.712363,0.776356,0.757416,0.891268
801,Reserve Malbec,2022.0,Trivento,3.7,6170.0,10.99,-0.333333,-1.235112,-0.458132,0.562400,0.902717,0.730521,0.769228,0.901779,0.764281,0.816012,0.754979,0.885567


In [335]:
user_input

{'meal': 'Costillas de Cerdo BBQ',
 'main_pairing': 'pork',
 'pairing_list': ['pork', 'spicy food'],
 'grape_list': ['Malbec', 'Pinot Noir'],
 'precio_min': 0,
 'precio_max': 23,
 'tastes': {'body': [np.float64(0.18189781021897816),
   np.float64(0.5168978102189782)],
  'tannins': [np.float64(0.15913370998116758),
   np.float64(0.30723701910142576)],
  'sweetness': [np.float64(0.0), np.float64(0.35640301318267426)],
  'acidity': [np.float64(0.42606403497492606),
   np.float64(0.5688568856885688)]},
 'weights': {'rating': 0.25,
  'price_quality': 0.25,
  'rating_qty': 0.1,
  'user_similarity': 0.3,
  'main_pairing': 0.1}}